# load_data

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Flatten, Embedding, LSTM, SpatialDropout1D, Input, Bidirectional,Dropout, Activation, GRU
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
train_data = pd.read_csv('Train_DataSet.csv')
train_label = pd.read_csv('Train_DataSet_Label.csv')
train = pd.merge(train_data, train_label, how='left', on='id')
train = train[(train.label.notnull()) & (train.content.notnull())]
test = pd.read_csv('Test_DataSet.csv')

train['title'] = train['title'].fillna('')
train['content'] = train['content'].fillna('')
test['title'] = test['title'].fillna('')
test['content'] = test['content'].fillna('')

Using TensorFlow backend.


# stop_special sign_word2vec

In [3]:
import re
def filter(text):
    text = re.sub("[A-Za-z0-9\!\=\？\%\[\]\,\（\）\>\<:&lt;\/#\. -----\_]", "", text)
    text = text.replace('图片', '')
    text = text.replace('\xa0', '') # 删除nbsp
    # new
    r1 =  "\\【.*?】+|\\《.*?》+|\\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\\\，。=？、：“”‘’￥……（）《》【】]"
    cleanr = re.compile('<.*?>')
    text = re.sub(cleanr, ' ', text)        #去除html标签
    text = re.sub(r1,'',text)
    text = text.strip()
    return text
def clean_text(data):
    data['title'] = data['title'].apply(lambda x: filter(x))
    data['content'] = data['content'].apply(lambda x: filter(x))
    return data
train = clean_text(train)
test = clean_text(test)
stop_words = pd.read_table('stop.txt', header=None)[0].tolist()
import jieba
import string
table = str.maketrans("","",string.punctuation)
def cut_text(sentence):
    tokens = list(jieba.cut(sentence))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens
train_title = [cut_text(sent) for sent in train.title.values]
train_content = [cut_text(sent) for sent in train.content.values]
test_title = [cut_text(sent) for sent in test.title.values]
test_content = [cut_text(sent) for sent in test.content.values]
all_doc = train_title + train_content + test_title + test_content

E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Possible set difference at position 43
  This is separate from the ipykernel package so we can avoid doing imports until
E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Possible set difference at position 45
  This is separate from the ipykernel package so we can avoid doing imports until
E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Possible set difference at position 46
  This is separate from the ipykernel package so we can avoid doing imports until
E:\anaconda\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 1.624 seconds.
Prefix dict has been built succesfully.


# textCNN

In [4]:
import gensim
import time
class EpochSaver(gensim.models.callbacks.CallbackAny2Vec):
    '''用于保存模型, 打印损失函数等等'''
    def __init__(self, save_path):
        self.save_path = save_path
        self.epoch = 0
        self.pre_loss = 0
        self.best_loss = 999999999.9
        self.since = time.time()

    def on_epoch_end(self, model):
        self.epoch += 1
        cum_loss = model.get_latest_training_loss() # 返回的是从第一个epoch累计的
        epoch_loss = cum_loss - self.pre_loss
        time_taken = time.time() - self.since
        print("Epoch %d, loss: %.2f, time: %dmin %ds" % 
                    (self.epoch, epoch_loss, time_taken//60, time_taken%60))
        if self.best_loss > epoch_loss:
            self.best_loss = epoch_loss
            print("Better model. Best loss: %.2f" % self.best_loss)
            model.save(self.save_path)
            print("Model %s save done!" % self.save_path)

        self.pre_loss = cum_loss
        self.since = time.time()

In [7]:
model_word2vec = gensim.models.Word2Vec(min_count=1, 
                                        window=5, 
                                        size=256,
                                        workers=4,
                                        batch_words=1000)
since = time.time()
model_word2vec.build_vocab(all_doc, progress_per=2000)
time_elapsed = time.time() - since
print('Time to build vocab: {:.0f}min {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
since = time.time()
model_word2vec.train(all_doc, total_examples=model_word2vec.corpus_count, 
                        epochs=5, compute_loss=True, report_delay=60*10,
                        callbacks=[EpochSaver('./final_word2vec_model')])
time_elapsed = time.time() - since
print('Time to train: {:.0f}min {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
#统计长度
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_title + test_title)
# 转化成词向量矩阵，利用新的word2vec模型
vocab_size = len(tokenizer.word_index)
print(vocab_size)
error_count=0
embedding_matrix = np.zeros((vocab_size + 1, 256))
for word, i in tqdm(tokenizer.word_index.items()):
    if word in model_word2vec:
        embedding_matrix[i] = model_word2vec.wv[word]
    else:
        error_count += 1
#句子的长度        
sequence = tokenizer.texts_to_sequences(train_title)
traintitle = pad_sequences(sequence, maxlen=30)
sequence = tokenizer.texts_to_sequences(test_title)
testtitle = pad_sequences(sequence, maxlen=30)

Time to build vocab: 0min 15s
Epoch 1, loss: 4106491.25, time: 0min 12s
Better model. Best loss: 4106491.25
Model ./final_word2vec_model save done!
Epoch 2, loss: 2704354.75, time: 0min 12s
Better model. Best loss: 2704354.75
Model ./final_word2vec_model save done!
Epoch 3, loss: 2281141.00, time: 0min 12s
Better model. Best loss: 2281141.00
Model ./final_word2vec_model save done!
Epoch 4, loss: 1962566.00, time: 0min 13s
Better model. Best loss: 1962566.00
Model ./final_word2vec_model save done!
Epoch 5, loss: 1864187.00, time: 0min 12s
Better model. Best loss: 1864187.00
Model ./final_word2vec_model save done!
Time to train: 1min 31s
32058


100%|█████████████████████████████████████████████████████████████████████████| 32058/32058 [00:00<00:00, 68942.16it/s]


In [8]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout
class TextCNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        # Embedding part can try multichannel as same as origin paper
        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen,
                              weights=[embedding_matrix])(input)
        convs = []
        for kernel_size in [3, 4, 5]:
            c = Conv1D(128, kernel_size, activation='relu')(embedding)
            c = GlobalMaxPooling1D()(c)
            convs.append(c)
        x = Concatenate()(convs)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model
    
model = TextCNN(maxlen=30, max_features=len(tokenizer.word_index) + 1,
                    embedding_dims=256, class_num=3, last_activation='softmax').get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy',metric_F1score])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 256)      8207104     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 28, 128)      98432       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 27, 128)      131200      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

In [9]:
import tensorflow as tf
def metric_F1score(y_true,y_pred):    
    TP=tf.reduce_sum(y_true*tf.round(y_pred))
    TN=tf.reduce_sum((1-y_true)*(1-tf.round(y_pred)))
    FP=tf.reduce_sum((1-y_true)*tf.round(y_pred))
    FN=tf.reduce_sum(y_true*(1-tf.round(y_pred)))
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    F1score=2*precision*recall/(precision+recall)
    return F1score

label = train['label'].astype(int)

train_X, val_X, train_Y, val_Y = train_test_split(traintitle, label, shuffle=True, test_size=0.2,
                                                    random_state=2019)
train_Y = to_categorical(train_Y)

model.fit(train_X,
          train_Y,
          batch_size=128,
          epochs=10)

Epoch 1/10
5812/5812 [==============================] - ETA: 1:45 - loss: 1.2944 - acc: 0.4062 - metric_F1score: 0.37 - ETA: 57s - loss: 1.5677 - acc: 0.3984 - metric_F1score: 0.3862 - ETA: 41s - loss: 1.3996 - acc: 0.4219 - metric_F1score: 0.407 - ETA: 33s - loss: 1.2955 - acc: 0.4668 - metric_F1score: 0.449 - ETA: 28s - loss: 1.2234 - acc: 0.5078 - metric_F1score: 0.487 - ETA: 24s - loss: 1.1924 - acc: 0.5247 - metric_F1score: 0.504 - ETA: 22s - loss: 1.1496 - acc: 0.5480 - metric_F1score: 0.528 - ETA: 20s - loss: 1.1062 - acc: 0.5684 - metric_F1score: 0.550 - ETA: 18s - loss: 1.0892 - acc: 0.5833 - metric_F1score: 0.565 - ETA: 17s - loss: 1.0917 - acc: 0.5914 - metric_F1score: 0.574 - ETA: 16s - loss: 1.0853 - acc: 0.5973 - metric_F1score: 0.580 - ETA: 15s - loss: 1.0731 - acc: 0.6009 - metric_F1score: 0.583 - ETA: 14s - loss: 1.0577 - acc: 0.6094 - metric_F1score: 0.593 - ETA: 13s - loss: 1.0497 - acc: 0.6116 - metric_F1score: 0.595 - ETA: 12s - loss: 1.0290 - acc: 0.6177 - metric_

5812/5812 [==============================] - ETA: 12s - loss: 0.0983 - acc: 0.9922 - metric_F1score: 0.988 - ETA: 11s - loss: 0.1177 - acc: 0.9883 - metric_F1score: 0.980 - ETA: 11s - loss: 0.1211 - acc: 0.9870 - metric_F1score: 0.981 - ETA: 11s - loss: 0.1193 - acc: 0.9863 - metric_F1score: 0.982 - ETA: 11s - loss: 0.1159 - acc: 0.9844 - metric_F1score: 0.981 - ETA: 10s - loss: 0.1121 - acc: 0.9857 - metric_F1score: 0.981 - ETA: 10s - loss: 0.1090 - acc: 0.9844 - metric_F1score: 0.981 - ETA: 10s - loss: 0.1049 - acc: 0.9863 - metric_F1score: 0.983 - ETA: 10s - loss: 0.1059 - acc: 0.9861 - metric_F1score: 0.983 - ETA: 9s - loss: 0.1104 - acc: 0.9836 - metric_F1score: 0.981 - ETA: 9s - loss: 0.1131 - acc: 0.9801 - metric_F1score: 0.97 - ETA: 9s - loss: 0.1142 - acc: 0.9798 - metric_F1score: 0.97 - ETA: 8s - loss: 0.1140 - acc: 0.9802 - metric_F1score: 0.97 - ETA: 8s - loss: 0.1155 - acc: 0.9794 - metric_F1score: 0.97 - ETA: 8s - loss: 0.1156 - acc: 0.9797 - metric_F1score: 0.97 - ETA: 8

5812/5812 [==============================] - ETA: 11s - loss: 0.0253 - acc: 0.9922 - metric_F1score: 0.992 - ETA: 11s - loss: 0.0251 - acc: 0.9961 - metric_F1score: 0.996 - ETA: 11s - loss: 0.0258 - acc: 0.9974 - metric_F1score: 0.997 - ETA: 11s - loss: 0.0272 - acc: 0.9961 - metric_F1score: 0.997 - ETA: 10s - loss: 0.0262 - acc: 0.9969 - metric_F1score: 0.997 - ETA: 10s - loss: 0.0301 - acc: 0.9961 - metric_F1score: 0.996 - ETA: 10s - loss: 0.0305 - acc: 0.9955 - metric_F1score: 0.996 - ETA: 10s - loss: 0.0338 - acc: 0.9951 - metric_F1score: 0.995 - ETA: 9s - loss: 0.0321 - acc: 0.9957 - metric_F1score: 0.996 - ETA: 9s - loss: 0.0306 - acc: 0.9961 - metric_F1score: 0.99 - ETA: 9s - loss: 0.0304 - acc: 0.9957 - metric_F1score: 0.99 - ETA: 9s - loss: 0.0307 - acc: 0.9954 - metric_F1score: 0.99 - ETA: 8s - loss: 0.0318 - acc: 0.9946 - metric_F1score: 0.99 - ETA: 8s - loss: 0.0308 - acc: 0.9950 - metric_F1score: 0.99 - ETA: 8s - loss: 0.0300 - acc: 0.9953 - metric_F1score: 0.99 - ETA: 8s 

In [10]:
from sklearn.metrics import f1_score
pred_val = model.predict(val_X)
print(f1_score(val_Y, np.argmax(pred_val, axis=1), average='macro'))

0.6118140168738982


In [ ]:
def cut_text(sentence):
    tokens = list(jieba.cut(sentence))
    #tokens = [token for token in tokens if token not in stop_words]
    return tokens

model_word2vec = gensim.models.Word2Vec(min_count=1, window=5,  size=256, workers=4,batch_words=1000)
model_word2vec.build_vocab(all_doc, progress_per=2000)
model_word2vec.train(all_doc, total_examples=model_word2vec.corpus_count, 
                        epochs=5, compute_loss=True, report_delay=60*10,
                        callbacks=[EpochSaver('./final')])